In [341]:
# Preparo entorno de ejecucion
import pandas as pd
import numpy as np
import re
# Configuro display de pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)

# Funciones auxiliares
NUMS = {"un":1, "dos":2, "tres":3,"cuatro":4,"cinco":5, "seis":6, "siete":7, "ocho":8, "nueve":9}

# showPerOfNull Imprime el porcentaje de de campos nulos de un data set
def showPerOfNull(df, show=True):
    nils = (df.isnull().sum()/df.shape[0]) * 100
    if show:
        print(nils)
    else:
        return nils

# fillAmountVar Recorre las filas de un data frame y rellena los campos nulos con el valor extraido de la columna descripcion a partir de una expresion regular
# 
# data_row: fila del data frame
# reg_exp: expresion regular
# col: columna del data frame
def fillAmountVar(data_row, reg_exp, col_input, col_output, reg_exp_group):
    matched = reg_exp.search(data_row[col_input])
    if matched is None:
        return data_row[col_output]
    lookFor = matched.group(reg_exp_group)
    if lookFor.isalpha():
        return NUMS[lookFor.lower()]
    if lookFor is not None:
        return lookFor
    return data_row[col_output]

# Objetivo Principal
### Armar un modelo de regresion para determinar el precio por metro cuadrado de un inmueble


### Preparacion de los datos
Antes de armar un modelo de regresion debemos preparar el data set vamos realizar un analsis exploratorio.

Vamos a eliminar columnas que no aportan informacion significativa
Entre estas tenemos: 
- 'properati_url' : Url en la base de properati
- 'image_thumbnail' : Url de la imagen de la primer foto
- 'geonames_id' : id de geonames

In [342]:

# Cargo data en data frame
df = pd.read_csv('properatti.csv', index_col=0)
print("Tamanio del data frame: ", df.shape[1], " columnas y ", df.shape[0], " filas \n")

# Eliminamos columnas que no nos interesan
try:
    df.drop(labels=['properati_url', 'image_thumbnail', 'geonames_id'], axis=1, inplace=True)
except:
    print("Ya no hay columnas properati_url, image_thumbnail, geonames_id")
print("Columnas restantes: ", df.shape[1])
for col in df.columns:
    print("->", col)

Tamanio del data frame:  25  columnas y  121220  filas 

Columnas restantes:  22
-> operation
-> property_type
-> place_name
-> place_with_parent_names
-> country_name
-> state_name
-> lat-lon
-> lat
-> lon
-> price
-> currency
-> price_aprox_local_currency
-> price_aprox_usd
-> surface_total_in_m2
-> surface_covered_in_m2
-> price_usd_per_m2
-> price_per_m2
-> floor
-> rooms
-> expenses
-> description
-> title


Vamos a tomar una muestra de 10 filas para comprender el problema

In [343]:
df.sample(3) 

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title
54098,sell,house,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,250000.0,USD,4411125.0,250000.0,215.0,215.0,1162.790698,1162.790698,NaN,NaN,NaN,"Muy linda casa de 2 plantas de gran calidad constructiva en lote interno de 640mts en Country Mapuche. En la P.B se ingresa al Living Comedor con hogar. Cocina con espacio para comedor diario. Escritorio independiente con acceso al toilette de la casa, lo cual lo convierte en una potencial habit...",Bustamante Propiedades | Mapuche en Venta
46506,sell,house,Martínez,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|Martínez|,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,750000.0,USD,13233375.0,750000.0,754.0,340.0,994.694960,2205.882353,NaN,NaN,NaN,"Excelente chalet inglÃ©s de estilo en dos plantas implantado sobre dos lotes (frente 17,32x 43,50 de fondo), totalmente original en pleno corazÃ³n de Martinez, a dos cuadras y media de Alvear, y del centro comercial. PrÃ³ximo a estaciÃ³n de tren y las Avenidas Santa Fe y Libertador. Importante h...","Casa 754m² con 2 Plantas en Muñiz 200, San Isidro, Martínez, por U$S 750.000"
19758,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,"-34.58522,-58.426652",-34.58522,-58.426652,114000.0,USD,2011473.0,114000.0,NaN,44.0,NaN,2590.909091,1.0,2.0,NaN,"Corredor Responsable: Mónica Silvia Martinuzzi - CUCICBA 5172Contacto: Verónica Zaltzman - MLS ID # 420621048-7Departamento de 2 amb de 44,6 m2 en excelente ubicación en Palermo Soho. A 3 cuadras de Plaza Italia y 3 de Juan B. Justo!!! Luminoso. Muy buena distribución. Hall de entrada, cocina in...",Venta 2 amb en inmejorable zona de Palermo Soho!


Vamos a ver que columnas se encuentran con valores nulos para luego poder completar los mismos con diferentes tecnicas de imputacion

In [344]:
# Extraigo las columnas que tienen valores nulos
df.isnull().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
description                        2
title                              0
dtype: int64

Podemos ver que los campos `operation`, `property_type`, `place_with_parent_names`, `country_name`, `state_name` y `title` no contien datos nulos.
Vamos a ver que porcentaje de las columnas nulas contienen datos nulos

In [345]:
df_null = df.isnull().sum()
df_null = df_null[df_null > 0]
rows = df.shape[0]
for col in df_null.index:
    por = "{0:.3f}".format((df_null[col]/rows) *100)
    print(f"-> {col} tiene {por} % valores nulos")

-> place_name tiene 0.019 % valores nulos
-> lat-lon tiene 42.526 % valores nulos
-> lat tiene 42.526 % valores nulos
-> lon tiene 42.526 % valores nulos
-> price tiene 16.837 % valores nulos
-> currency tiene 16.838 % valores nulos
-> price_aprox_local_currency tiene 16.837 % valores nulos
-> price_aprox_usd tiene 16.837 % valores nulos
-> surface_total_in_m2 tiene 32.443 % valores nulos
-> surface_covered_in_m2 tiene 16.422 % valores nulos
-> price_usd_per_m2 tiene 43.395 % valores nulos
-> price_per_m2 tiene 27.687 % valores nulos
-> floor tiene 93.484 % valores nulos
-> rooms tiene 60.906 % valores nulos
-> expenses tiene 88.235 % valores nulos
-> description tiene 0.002 % valores nulos


Vamosa imprimir una muestra de cada uno de los campos que contiene campos vacios

In [346]:
for field in df_null.index:
    print(f'{field} : {df[field][np.random.randint(0, df.shape[0])]}')


place_name : Mar del Plata
lat-lon : nan
lat : nan
lon : nan
price : 178000.0
currency : nan
price_aprox_local_currency : 6528465.0
price_aprox_usd : 135000.0
surface_total_in_m2 : 38.0
surface_covered_in_m2 : 360.0
price_usd_per_m2 : 1588.235294117647
price_per_m2 : 3404.255319148936
floor : nan
rooms : nan
expenses : nan
description : Edificio Pandora; Departamento 2 ambientes a 200 metros del mar.Edificio Pandora; Departamento 2 ambientes ubicado en el Paseo 132 y Av 2 bis (calle cortada, con articulado) excelente ubicación 1 habitación matrimonial, 1 baño y cocina comedor.


## Datos faltantes
Tenemos distintas maneras de tratar los datos faltantes, vamos a ir analizando en cada uno de estas columnas que metodo se ajusta mejor

### `country_name`
Esta columna no aporta ningun valor, ya que todas las filas contienen el mismo valor "Argentina"

In [347]:
duplicated = df.country_name.duplicated().sum()
total = df.country_name.size
print(f'{duplicated} duplicados de {total}')
try:
    df.drop(labels=['country_name'], axis=1, inplace=True)
except:
    print("Ya no hay columnas country_name")
print("Columnas restantes: ", df.shape[1])

121219 duplicados de 121220
Columnas restantes:  21


### `place_name` & `description`
La columna `place_name` con datos nulos representa el 0.019 % y `description` el 0.002% , en este caso vamos a eliminar las filas nulas directamente.

In [348]:
df.dropna(
    axis=0,
    how='any',
    subset=["description", "place_name"],
    inplace=True
)
# Datos nulos:
showPerOfNull(df)

operation                      0.000000
property_type                  0.000000
place_name                     0.000000
place_with_parent_names        0.000000
state_name                     0.000000
lat-lon                       42.533933
lat                           42.533933
lon                           42.533933
price                         16.840629
currency                      16.841454
price_aprox_local_currency    16.840629
price_aprox_usd               16.840629
surface_total_in_m2           32.444408
surface_covered_in_m2         16.424770
price_usd_per_m2              43.397830
price_per_m2                  27.691736
floor                         93.482404
rooms                         60.904328
expenses                      88.232188
description                    0.000000
title                          0.000000
dtype: float64


### `lat-lon`, `lat` & `lon`
Estas columnas contienen 42.534 % de sus filas vacios, no podemos utilizar esto datos para un estudio por lo tanto vamos a eliminar las columnas directamente

In [349]:
try:
    df.drop(labels=['lat-lon', 'lat', 'lon'], axis=1, inplace=True)
except:
    print("Ya no hay columnas lat-lon, lat, lon")
# Datos nulos:
showPerOfNull(df)

operation                      0.000000
property_type                  0.000000
place_name                     0.000000
place_with_parent_names        0.000000
state_name                     0.000000
price                         16.840629
currency                      16.841454
price_aprox_local_currency    16.840629
price_aprox_usd               16.840629
surface_total_in_m2           32.444408
surface_covered_in_m2         16.424770
price_usd_per_m2              43.397830
price_per_m2                  27.691736
floor                         93.482404
rooms                         60.904328
expenses                      88.232188
description                    0.000000
title                          0.000000
dtype: float64


### `rooms`
La columna `rooms` tiene 60.904 % de sus datos nulos

Para poder imputar sus datos vamos a extraerlos a partir de la columna `descripcion` & `title` utilizando expresiones regulares


In [350]:

exp= r"([0-9]|(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve))+\s(hab|amb|dor|pieza)+"
roomExp =re.compile(exp, re.MULTILINE | re.IGNORECASE)

print("Datos nulos antes de rellenar 'rooms': " +"{0:.4f}".format(showPerOfNull(df.rooms, show=False)) + "%")
# Aplico funcion de imputacion
df["rooms"] = df.apply(lambda x: fillAmountVar(data_row=x, reg_exp=roomExp, col_input="description", col_output="rooms", reg_exp_group=1), axis=1)
df["rooms"] = df.apply(lambda x: fillAmountVar(data_row=x, reg_exp=roomExp, col_input="title", col_output="rooms", reg_exp_group=1), axis=1)

print("Datos nulos despues de rellenar 'rooms:'" +"{0:.4f}".format(showPerOfNull(df.rooms, show=False)) + "%")
showPerOfNull(df)

Datos nulos antes de rellenar 'rooms': 60.9043%
Datos nulos despues de rellenar 'rooms:'15.8579%
operation                      0.000000
property_type                  0.000000
place_name                     0.000000
place_with_parent_names        0.000000
state_name                     0.000000
price                         16.840629
currency                      16.841454
price_aprox_local_currency    16.840629
price_aprox_usd               16.840629
surface_total_in_m2           32.444408
surface_covered_in_m2         16.424770
price_usd_per_m2              43.397830
price_per_m2                  27.691736
floor                         93.482404
rooms                         15.857915
expenses                      88.232188
description                    0.000000
title                          0.000000
dtype: float64


### `floor`
La columna `floor` tiene 93.482 % de sus datos nulos

Para poder imputar sus datos vamos a extraerlos a partir de la columna `descripcion` & `title` utilizando expresiones regulares

In [351]:

# exp= r"([0-9]|(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve))+\s(hab|amb|dor|pieza)+"
# floorExp =re.compile(exp, re.MULTILINE | re.IGNORECASE)

# print("Datos nulos antes de rellenar 'floor': " +"{0:.4f}".format(showPerOfNull(df.floor, show=False)) + "%")
# # Aplico funcion de imputacion
# df["floor"] = df.apply(lambda x: fillAmountVar(data_row=x, reg_exp=floorExp, col_input="description", col_output="floor", reg_exp_group=1), axis=1)
# df["floor"] = df.apply(lambda x: fillAmountVar(data_row=x, reg_exp=floorExp, col_input="title", col_output="floor", reg_exp_group=1), axis=1)

# print("Datos nulos despues de rellenar 'floor:'" +"{0:.4f}".format(showPerOfNull(df.rooms, show=False)) + "%")
# showPerOfNull(df)